In [132]:
from utils.read_file import *

In [133]:
QUESTION_PATH = "競賽資料集/dataset/preliminary/questions_example.json"

FAQ_PATH = "競賽資料集/reference/faq/pid_map_content.json"
ANS_PATH = "競賽資料集/dataset/preliminary/ground_truths_example.json"

In [134]:
faq_data = read_json(FAQ_PATH)
question_data = read_json(QUESTION_PATH)["questions"]
#ans = read_json(ANS_PATH)["ground_truths"]

In [135]:
from utils.read_file import read_target_faq

In [136]:
from rank_bm25 import BM25Okapi
import numpy as np
import jieba
def get_ans_bm25(question_data: dict, documents: list, k=1):
    tokenized_documents = [list(jieba.cut(doc)) for doc in documents]
    bm25 = BM25Okapi(tokenized_documents)
    
    user_query = list(jieba.cut(question_data["query"]))
    scores = bm25.get_scores(user_query)
    
    k_highest = np.argsort(scores)[-k:][::-1]
    return k_highest, np.sort(scores)[::-1][:k]

In [148]:
def validate_faq(question_data):
    ans = read_json(ANS_PATH)["ground_truths"]
    correct = 0
    half_correct = 0

    for i in range(100, 150):
        #print(ans[i]["retrieve"])
        src = question_data[i]["source"]
        #print(question_data[i]["qid"])
        #print(src)
        document_texts= read_target_faq(faq_data, src)
        k_highest, similarities = get_ans_bm25(question_data[i], document_texts, k=1)
        
        # 取得最相關的 FAQ 文檔索引
        top_faq_id = src[k_highest[0]]

        if top_faq_id == ans[i]["retrieve"]:
            correct += 1
        elif any(faq_id == ans[i]["retrieve"] for faq_id in k_highest[:3]):
            half_correct += 1
        if top_faq_id != ans[i]["retrieve"]:
            print(i+1)
            print(f"faq_id {top_faq_id}")
    # 輸出準確率統計結果
    total_questions = 50  # 驗證的問題數量: 50
    print(f"正確回答數量: {correct}")
    print(f"部分正確回答數量: {half_correct}")
    print(f"總回答數量: {total_questions}")
    print(f"準確率: {correct / total_questions:.2%}")
    print(f"部分準確率: {half_correct / total_questions:.2%}")


In [149]:
validate_faq(question_data)

109
faq_id 409
128
faq_id 293
135
faq_id 431
144
faq_id 542
148
faq_id 420
正確回答數量: 45
部分正確回答數量: 0
總回答數量: 50
準確率: 90.00%
部分準確率: 0.00%
